<a href="https://colab.research.google.com/github/TahaMsv/Bachelor-Project/blob/main/Bachelor_project(Pre_processing).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install mne

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 46.8 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [ ]:
import os
import mne
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
# base_path = "/drive/Shared with me/Bachelor's project/UC San Diego/"
base_path = "/content/drive/MyDrive/Bachelor's project/UC San Diego/"

#Pre-Processing

In [ ]:
import os
import numpy as np
import mne
from sklearn.model_selection import train_test_split
from scipy.signal import butter, lfilter
from scipy import signal
from scipy.ndimage import zoom


data_folder_normal = base_path + "hc"
data_folder_patient = base_path + "pd"

# Load EEG data from folders
def load_eeg_data(folder):
    data = []
    for file in os.listdir(folder):
        if file.endswith('.bdf'):
            file_path = os.path.join(folder, file)
            raw = mne.io.read_raw_bdf(file_path, preload=True)
            eeg_data = raw.get_data()
            data.append(eeg_data)
    return data

# Load data from both folders
normal_data = load_eeg_data(data_folder_normal)
patient_data = load_eeg_data(data_folder_patient)

Extracting EDF parameters from /content/drive/MyDrive/Bachelor's project/UC San Diego/hc/Copy of sub-hc1_ses-hc_task-rest_eeg.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 98303  =      0.000 ...   191.998 secs...
Extracting EDF parameters from /content/drive/MyDrive/Bachelor's project/UC San Diego/hc/Copy of sub-hc4_ses-hc_task-rest_eeg.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 92671  =      0.000 ...   180.998 secs...
Extracting EDF parameters from /content/drive/MyDrive/Bachelor's project/UC San Diego/hc/Copy of sub-hc2_ses-hc_task-rest_eeg.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 99327  =      0.000 ...   193.998 secs...
Extracting EDF parameters from /content/drive/MyDrive/Bachelor's project/UC San Diego/hc/Copy of sub-hc7_ses-hc_task-rest_eeg.bdf...
BDF file detected
Setting channel info structure...

In [ ]:
min_length = min([eeg_data.shape[1] for eeg_data in normal_data + patient_data])  #Minimum length of EEG signals

# Resize EEG data to the minimum length
def resize_eeg_data(data, new_length):
    resized_data = []
    for eeg_data in data:
        resized_eeg_data = zoom(eeg_data, (1, new_length / eeg_data.shape[1]), order=3)
        resized_data.append(resized_eeg_data)
    return resized_data

# Resize data from both groups
resized_normal_data = resize_eeg_data(normal_data, min_length)
resized_patient_data = resize_eeg_data(patient_data, min_length)

In [ ]:
sampling_rate = 512  # Sampling rate of EEG data
window_length = 2 * sampling_rate  # 2-second window length

# Segment data into 2-second windows
def segment_data(data, window_length):
    segments = []
    for eeg_data in data:
        num_segments = eeg_data.shape[1] // window_length
        for i in range(num_segments):
            segment = eeg_data[:, i * window_length : (i + 1) * window_length]
            segments.append(segment)
    return segments

# Segment data from both groups
normal_segments = segment_data(resized_normal_data, window_length)
patient_segments = segment_data(resized_patient_data, window_length)

# Apply thresholding and bandpass filtering
def preprocess_segment(segment):
    # Apply thresholding to discard signal amplitudes exceeding ± 100 µV
    threshold = 100  # in µV
    segment[segment > threshold] = threshold
    segment[segment < -threshold] = -threshold

    # Apply a 6th-order bandpass Butterworth filter (1-49 Hz)
    lowcut = 1  # in Hz
    highcut = 49  # in Hz
    nyquist = 0.5 * sampling_rate
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(6, [low, high], btype='band')
    filtered_segment = lfilter(b, a, segment, axis=1)

    return filtered_segment

# Apply preprocessing to both groups
preprocessed_normal = [preprocess_segment(segment) for segment in normal_segments]
preprocessed_patient = [preprocess_segment(segment) for segment in patient_segments]

# Prepare labels
labels_normal = np.zeros(len(preprocessed_normal))  # 0 for normal
labels_patient = np.ones(len(preprocessed_patient))  # 1 for patients

# Combine data and labels
data = np.concatenate((preprocessed_normal, preprocessed_patient), axis=0)
labels = np.concatenate((labels_normal, labels_patient), axis=0)

In [ ]:
print(data.shape)
print(labels.shape)

(2880, 41, 1024)
(2880,)


In [ ]:
np.save(base_path + "pre processed data.npy", data)
np.save(base_path + "pre processed labels.npy", labels)